# 1. 工具包导入

In [1]:
import cv2
import time
import keras
import numpy as np
import fpga_drive
import matplotlib.pyplot as plt
from pynq import Overlay
from keras.applications.inception_v3 import preprocess_input
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

Using Theano backend.


# 2. 准备图像

In [2]:
num_classes = 8
quant_scale = 256
classlist = ['A2C', 'A3C', 'A4C', 'ASA', 'BSA', 'MSA', 'PLA', 'PSA']

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=127
)
val_generator = val_datagen.flow_from_directory(directory='data',
                                                target_size=(256, 256),
                                                color_mode='grayscale',
                                                batch_size=512,
                                                shuffle=False)
[x_test, y_test] = val_generator[4]
print('x_test shape:', x_test.shape)
print(x_test.shape[0], 'test samples')

Found 7545 images belonging to 8 classes.
x_test shape: (512, 256, 256, 1)
512 test samples


# 3. 读取网络模型

In [3]:
model = load_model('ultrasound.h5')

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 63, 63, 16)        416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 32)          12832     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 5, 5, 32)          18464     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 16)          4624      
_________________________________________________________________
flatten_1 (Flatten)          (None, 144)               0         
__________

# 4. 配置FPGA Overlay

In [5]:
overlay = Overlay("ultra.bit")

# 5. 配置HDMI端口

In [6]:
overlay.video.configure(800, 600, 24)

# 6. 加载网络权值

In [7]:
weight = model.get_layer('conv2d_1').get_weights()
overlay.cnn.load_weight_conv(weight, 1, quant_scale, 0.95)
weight = model.get_layer('conv2d_2').get_weights()
overlay.cnn.load_weight_conv(weight, 2, quant_scale, 1.25)
weight = model.get_layer('conv2d_3').get_weights()
overlay.cnn.load_weight_conv(weight, 3, quant_scale, 1.6)
weight = model.get_layer('conv2d_4').get_weights()
overlay.cnn.load_weight_conv(weight, 4, quant_scale, 1.3)
weight = model.get_layer('conv2d_5').get_weights()
overlay.cnn.load_weight_conv(weight, 5, quant_scale, 1.75)
weight = model.get_layer('dense_1').get_weights()
overlay.cnn.load_weight_fc(weight, 6, quant_scale, 1.6)
weight = model.get_layer('dense_2').get_weights()
overlay.cnn.load_weight_fc(weight, 7, quant_scale, 1)

# 7. 运算一个Batch

In [8]:
count = 0
batch_size = 100
start_time = time.process_time()
result = overlay.cnn.execute(x_test, batch_size, 1, 256, 8, 1)
end_time = time.process_time()
print("FPAG Elapsed Test Time: ", (end_time-start_time)*1000, "ms")
for i in range(batch_size):
    if result[i].argmax() == y_test[i].argmax():
        count = count + 1
score = count/batch_size
print('FPGA accuracy:', score)

Elapsed Test Time:  252.59957799998745 ms
FPAG Elapsed Test Time:  852.3827820000065 ms
FPGA accuracy: 1.0


In [9]:
count = 0
batch_size = 100
start_time = time.process_time()
result = model.predict(x_test[0:batch_size])
end_time = time.process_time()
print("Arm Elapsed Test Time: ", (end_time-start_time)*1000, "ms")
for i in range(batch_size):
    if result[i].argmax() == y_test[i].argmax():
        count = count + 1
score = count/batch_size
print('Arm accuracy:', score)

Arm Elapsed Test Time:  6716.667694999998 ms
Arm accuracy: 1.0


# 循环展示

In [10]:
[x_test, y_test] = val_generator[4]

In [ ]:
batch_size = 512
for index in range(batch_size):
    result = overlay.cnn.execute_s(x_test[index], 1, 256, 8, 1)
    img = x_test[index] + 127
    img = cv2.resize(img, (800, 600)).astype(np.uint8)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.putText(img, classlist[result.argmax()], (600, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)
    overlay.video.display(img)

In [12]:
batch_size = 512
for index in range(batch_size):
    result = overlay.cnn.execute_s(x_test[index], 1, 256, 8, 1)
    img = x_test[index] + 127
    img = cv2.resize(img, (800, 600)).astype(np.uint8)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.putText(img, classlist[y_test[index].argmax()], (600, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)
    overlay.video.display(img)

# 测试网络全局准确率

In [14]:
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=127
)
val_generator = val_datagen.flow_from_directory(directory='data',
                                                target_size=(256, 256),
                                                color_mode='grayscale',
                                                batch_size=256,
                                                shuffle=False)

Found 7545 images belonging to 8 classes.


In [15]:
count = 0
for j in range(29):
    [x_test, y_test] = val_generator[j]
    batch_size = x_test.shape[0]
    result = overlay.cnn.execute(x_test, batch_size, 1, 256, 8, 1)
    for i in range(batch_size):
        if result[i].argmax() == y_test[i].argmax():
            count = count + 1
score = count/7545
print('FPGA accuracy:', score)

Elapsed Test Time:  572.1692669999925 ms
Elapsed Test Time:  489.31970399999614 ms
Elapsed Test Time:  641.0400209999807 ms
Elapsed Test Time:  640.8378200000016 ms
Elapsed Test Time:  640.9172330000388 ms
Elapsed Test Time:  640.904565000028 ms
Elapsed Test Time:  640.8796729999722 ms
Elapsed Test Time:  640.954445000034 ms
Elapsed Test Time:  640.8238629999801 ms
Elapsed Test Time:  640.8981339999968 ms
Elapsed Test Time:  640.8398140000031 ms
Elapsed Test Time:  640.9148310000319 ms
Elapsed Test Time:  640.8896570000024 ms
Elapsed Test Time:  640.7979869999849 ms
Elapsed Test Time:  640.9772089999706 ms
Elapsed Test Time:  641.0089770000127 ms
Elapsed Test Time:  640.9293680000019 ms
Elapsed Test Time:  640.6825860000254 ms
Elapsed Test Time:  639.9308109999993 ms
Elapsed Test Time:  640.6457249999562 ms
Elapsed Test Time:  640.7502019999924 ms
Elapsed Test Time:  640.7829030000016 ms
Elapsed Test Time:  640.7291030000124 ms
Elapsed Test Time:  640.9000099999957 ms
Elapsed Test Time

In [16]:
score = model.evaluate_generator(val_generator, verbose=1)
score[1]

30/30 [==============================] - 557s 19s/step


0.8160371107996656

In [ ]:
# 设置单层权值
# x=[1,2,3,4,5,6,7,8,9]
# x = np.transpose(x)
# x=np.reshape(x,(1,3,3,1,1))
# model.get_layer('conv2d_1').set_weights(x)

In [ ]:
# 获取中间结果
# intermediate_layer_model = Model(inputs=model.input,
#                                  outputs=model.get_layer('conv2d_1').output)
# pre = intermediate_layer_model.predict(x_test[0].reshape(1,256,256,1))

In [ ]:
# 展示图片
# result = np.reshape(x_test[0], (256,256))
# plt.figure(figsize=(4, 4))
# plt.imshow(result, 'gray')